In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import scipy.optimize as opt
from scipy.integrate import odeint, quad
from scipy.interpolate import interp1d
from scipy.optimize import curve_fit

from pathlib import Path
import os
import sys
from byutpl.properties import water as water

In [ ]:
# ----- Constants ----- #
# heat exchanger physical parameters
di = .206 * 2.54 / 100                  # m
do = .25 * 2.54 / 100                   # m
L = 14 * 2.54 / 100                     # m
k = 13.4                                # W/m.K 316 SS
g = 9.81                                # m/s^2
N = 56                                  # number of tubes    

# calculate the heat transfer area
Ai = .25 * np.pi * di**2
Ao = .25 * np.pi * do**2

# ----- Functions ----- #
def hi(Qi,Ti):
    # calculate the velocity
    v = Qi / Ai

    # calculate the Reynolds number
    Re = water.ldn(Ti) * v * di / water.lvs(Ti)

    # calculate the Nusselt number
    if Re < 10000:
        Nu = 3.66
    else:
        Nu = .023 * (Re**.8) * water.lpr(Ti)**.4

    # calculate heat transfer coefficient
    h = Nu * water.ltc(Ti) / di
    return h

def ho(Ps,Ts):
    # pull in the values
    rhol = water.ldn(Ts)
    rhov = water.vdn(Ts,Ps)
    kl = water.ltc(Ts)
    mul = water.lvs(Ts)
    Tsat = water.tsat(Ps)
    cpl = water.lcp(Ts) / water.mw
    hfg = water.hvp(Ts) / water.mw

    # find Ja
    Ja = cpl * (Tsat - Ts) / hfg

    # calculate the condensation energy
    hfp = hfg * (1 + (.68 * Ja))

    # calculate the heat transfer coefficient
    h = .729 * (rhol * g * (rhol - rhov) * hfp * kl**3 / (N * mul * (Tsat - Ts) * do))**.25
    return h

def model(inputs,Rf):
    Qw,Ps,Twi,Two = inputs

    # find the average temperature of water in the heat exchanger for accurate properties
    Tweff = (Twi + Two) / 2

    #                       |                 |                                 |
    #      convection_inner | fouling_inner   |             conduction          | convection_outer
    #                       |                 |                                 |
    UA = (hi(Qw, Tweff) * Ai)**-1 + (Rf / Ai) + (np.log(do / di) / (2 * np.pi * k * L)) + (ho(Ps, Tweff) / Ao)**-1
    return UA




# correlation for if we want to try to fit both fouling factors

# Qi,Qo = Qs
# #                       |                 |                                  |                 |
# #      convection_inner | fouling_inner   |             conduction           |  fouling_outer  | convection_outer
# #                       |                 |                                  |                 |
# sumR = (hi(Qi) * Ai)**-1 + (Rfi / Ai) + (np.log(do / di) / (2 * np.pi * k * L)) + (Rfo / Ao) + (ho(Qo) / Ao)**-1

$$UA = (\Sigma R)^-1 = \frac{1}{h_i(\dot V)A_i} + \frac{R_{f,i}^"}{A_i} + \frac{ln(d_o / d_i)}{2\pi kL}+ \frac{R_{f,o}^"}{A_o} + \frac{1}{h_o(\dot V)A_o}$$

In [2]:
help(water)

Help on module byutpl.properties.water in byutpl.properties:

NAME
    byutpl.properties.water

DESCRIPTION
    This library contains functions for the properties of water.
    The values come from the DIPPR(R) Sample database [1],
    and the DIPPR(R) abbreviations are used. Vapor properties that are
    dependent on pressure are obtained using the Soave-Redlich-Kwong equation
    of state.

    This module is part of the byutpl package. Import the module using

      import byutpl.properties.water as wtr

    When imported in this way, constant properties can be called as

      wtr.acen

    which returns the acentric factor. Temperature dependent properties
    can be called as

      wtr.vtc(t)

    which returns the vapor thermal conductivity at `t` where `t` is
    temperature in units of K. Temperature and pressure dependent properties
    can be called as

      wtr.vcp(t,p)

    which returns the vapor heat capacity at `t` and `p` where
    `t` is temperature in units of K an